In [2]:
import nest_asyncio
nest_asyncio.apply()


# Initialize the OpenAI client with the KitchenAI control plane

Make sure your control plane is running at localhost:8001


In [3]:
from openai import OpenAI
from whisk.kitchenai_sdk.http_schema import ChatExtraBody

client = OpenAI(base_url="http://localhost:8001/v1")

# Create a `FileExtraBody` object with metadata
- Assign a unique `client_id`
- Define a namespace, label, and version
- Attach metadata (e.g., user_id)
Upload the file (`README.md`) to KitchenAI for processing

> NOTE: at this point you can look at the bento.ipynb to see your storage function process the file. (It must be actively running!)


In [9]:
import json

from whisk.kitchenai_sdk.http_schema import FileExtraBody   

file_extra_body = FileExtraBody(
    client_id="61737fe4-d76c-4146-8d48-58a7b923ab78",
    namespace="my-remote-client",
    label="storage",
    version="1.0.0",
    metadata="user_id=123,other_key=value"  # Changed to string format
)

response = client.files.create(
    file=open("README.md", "rb"),
    purpose="chat",
    extra_body=file_extra_body.model_dump()
)
print(response)

FileObject(id='39', bytes=23, created_at=1738870798, filename='README.md', object='file', purpose='{"client_id": "61737fe4-d76c-4146-8d48-58a7b923ab47", "namespace": "my-remote-client", "label": "storage", "version": "0.0.1", "metadata": {"user_id": "123", "other_key": "value"}}', status='pending', status_details=None)


# List Files 

list the existing files 

In [5]:
c_list  = client.files.list()

print(c_list)

c_list.data[0].id

SyncCursorPage[FileObject](data=[FileObject(id='40', bytes=176299, created_at=1738876870, filename='consultancy-rfp.pdf', object='file', purpose='{"client_id": "61737fe4-d76c-4146-8d48-58a7b923ab47", "namespace": "my-remote-client", "label": "storage", "version": "0.0.1", "metadata": {}}', status='completed', status_details=None), FileObject(id='41', bytes=168661, created_at=1738924999, filename='advertising-rfp.pdf', object='file', purpose='{"client_id": "61737fe4-d76c-4146-8d48-58a7b923ab47", "namespace": "my-remote-client", "label": "storage", "version": "0.0.1", "metadata": {}}', status='completed', status_details=None)], object='list')


'40'

In [6]:
client.files.retrieve(c_list.data[0].id)


FileObject(id='40', bytes=176299, created_at=1738876870, filename='consultancy-rfp.pdf', object='file', purpose='{"client_id": "61737fe4-d76c-4146-8d48-58a7b923ab47", "namespace": "my-remote-client", "label": "storage", "version": "0.0.1", "metadata": {}}', status='completed', status_details=None)

## Query with no RAG

First, let's test querying without RAG (Retrieval Augmented Generation).
This means the model will respond based on its pre-trained knowledge,
without referencing any of our uploaded documents.
We'll use the 'query-no-rag' endpoint for this test.

In [7]:


chat_extra_body = ChatExtraBody(
    namespace="my-remote-client",
    version="1.0.0",
)   

print(chat_extra_body.model_dump())

response = client.chat.completions.create(
    model="@61737fe4-d76c-4146-8d48-58a7b923ab78/query-no-rag", #insert your client ID
    messages=[{"role": "user", "content": "whats the most important part of the readme?"}],
    metadata={"user_id": "123"},
    extra_body=chat_extra_body.model_dump()
)

print(response)


{'namespace': 'my-remote-client', 'version': '1.0.0'}
ChatCompletion(id='98ec82e2-5186-47c1-bca8-c769d606c8f9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The most important part of a README file is typically the introduction or overview section. This section should provide a brief summary of what the project is, what problem it solves, and how it can be used. It should also include any important information or instructions that users need to know before getting started with the project. Additionally, the README should include any necessary installation or setup instructions, as well as any dependencies or requirements that need to be met in order to use the project successfully.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, retrieval_context=None), error=None)], created=1738950191, model='@61737fe4-d76c-4146-8d48-58a7b923ab78/query-no-rag', object='chat.completion', service_tier=None, system

In [8]:
response.choices[0].message.content

'The most important part of a README file is typically the introduction or overview section. This section should provide a brief summary of what the project is, what problem it solves, and how it can be used. It should also include any important information or instructions that users need to know before getting started with the project. Additionally, the README should include any necessary installation or setup instructions, as well as any dependencies or requirements that need to be met in order to use the project successfully.'

# Query with RAG

Now let's test querying with RAG (Retrieval Augmented Generation).
This means the model will respond based on both its pre-trained knowledge
and the content of our uploaded documents.
We'll use the 'query' endpoint for this test.

In [9]:
chat_extra_body = ChatExtraBody(
    namespace="my-remote-client",
    version="1.0.0",
)   

print(chat_extra_body.model_dump())

response = client.chat.completions.create(
    model="@61737fe4-d76c-4146-8d48-58a7b923ab78/query", #insert your client ID
    messages=[{"role": "user", "content": "whats the most important part of the readme?"}],
    metadata={"user_id": "123"},
    extra_body=chat_extra_body.model_dump()
)

print(response)

{'namespace': 'my-remote-client', 'version': '1.0.0'}
ChatCompletion(id='8e0b297f-22db-4de0-9c51-509420f572e2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The most important part of the README is typically the content that provides a clear and concise overview of what the document covers, its purpose, and any key information that users need to know. This section often sets the tone for the rest of the document and helps users quickly understand the main points without having to read through the entire text.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, retrieval_context=[{'text': 'THIS IS A DEMO README', 'metadata': {'user_id': '123', 'other_key': 'value', 'document_title': '"Demo README Overview: A Comprehensive Guide to Understanding the Demo README"', 'questions_this_excerpt_can_answer': '1. What is the user_id associated with the document?\n2. What is the title of the document and what do

In [13]:
client.files.delete(c_list.data[0].id)


FileDeleted(id='39', deleted=True, object='file')